In [1]:
import pandas as pd
from sqlalchemy import create_engine

In [2]:
#Importing Wine CSV File
csv_file = "Resources/winemag-data_first150k.csv"
winedata_csv = pd.read_csv(csv_file)
winedata_csv.head()

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,variety,winery
0,0,US,This tremendous 100% varietal wine hails from ...,Martha's Vineyard,96,235.0,California,Napa Valley,Napa,Cabernet Sauvignon,Heitz
1,1,Spain,"Ripe aromas of fig, blackberry and cassis are ...",Carodorum Selección Especial Reserva,96,110.0,Northern Spain,Toro,NaN,Tinta de Toro,Bodega Carmen Rodríguez
2,2,US,Mac Watson honors the memory of a wine once ma...,Special Selected Late Harvest,96,90.0,California,Knights Valley,Sonoma,Sauvignon Blanc,Macauley
3,3,US,"This spent 20 months in 30% new French oak, an...",Reserve,96,65.0,Oregon,Willamette Valley,Willamette Valley,Pinot Noir,Ponzi
4,4,France,"This is the top wine from La Bégude, named aft...",La Brûlade,95,66.0,Provence,Bandol,NaN,Provence red blend,Domaine de la Bégude


In [3]:
#Importing Wine JSON File
json_file = "Resources/winemag-data-130k-v2.json"
winedata_json = pd.read_json(json_file)
winedata_json.head()

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,None,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,None,None,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,US,"Tart and snappy, the flavors of lime flesh and...",None,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,None,Alexander Peartree,None,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks


In [18]:
# Cleaning & Combining Wine Dataframes
cols = ['country', 'province', 'points', 'region_1', 
        'region_2', 'variety', 'winery', 'price']

clean_csv = winedata_csv[cols]
clean_json = winedata_json[cols]

main_cols = ['country', 'points', 'price', 'variety', 'region_1', 'region_2']

wine_df = clean_csv.merge(clean_json)

wine_df = wine_df.dropna(subset = main_cols)

rename_cols = {'country':'Country', 'province':'Province', 'description':'Description',
               'points':'Points', 'region_1':'Region_1', 'region_2':'Region_2', 
               'variety':'Variety', 'winery':'Winery', 'price':'Price' }

wine_df = wine_df.rename(columns=rename_cols)

wine_df.head()

,Country,Province,Points,Region_1,Region_2,Variety,Winery,Price
1,US,Oregon,95,Chehalem Mountains,Willamette Valley,Pinot Noir,Bergström,65.0
2,US,California,95,Diamond Mountain District,Napa,Cabernet Sauvignon,Hall,325.0
5,US,Oregon,94,Willamette Valley,Willamette Valley,Pinot Noir,Ponzi,105.0
6,US,Oregon,94,Willamette Valley,Willamette Valley,Pinot Noir,Ponzi,105.0
7,US,Oregon,94,Willamette Valley,Willamette Valley,Pinot Noir,Ponzi,105.0


In [19]:
# Importing Country Data
xlsx_file = "./Resources/country_data.xlsx"
country_df = pd.read_excel(xlsx_file)

country_df.head()

,Country,Capital,Continent,Region,Primary Language,Secondary Language
0,US,Washington D.C.,North America,NAM,English,Spanish
1,Italy,Rome,Europe,EMEA,Italian,Catalan
2,France,Paris,Europe,EMEA,French,Basque
3,Spain,Madrid,Europe,EMEA,Spanish,Catalan
4,Chile,Santiago,South America,LATAM,Spanish,Spanish


In [26]:
# Only wine data for countries in country file

countries = country_df['Country'].unique().tolist()
countries

wine_df = wine_df[wine_df['Country'].isin(countries)]

In [27]:
# Merging wine and country data frames

wine_country = pd.merge(wine_df, country_df, on='Country', how='left')

wine_country.head()

,Country,Province,Points,Region_1,Region_2,Variety,Winery,Price,Capital,Continent,Region,Primary Language,Secondary Language
0,US,Oregon,95,Chehalem Mountains,Willamette Valley,Pinot Noir,Bergström,65.0,Washington D.C.,North America,NAM,English,Spanish
1,US,California,95,Diamond Mountain District,Napa,Cabernet Sauvignon,Hall,325.0,Washington D.C.,North America,NAM,English,Spanish
2,US,Oregon,94,Willamette Valley,Willamette Valley,Pinot Noir,Ponzi,105.0,Washington D.C.,North America,NAM,English,Spanish
3,US,Oregon,94,Willamette Valley,Willamette Valley,Pinot Noir,Ponzi,105.0,Washington D.C.,North America,NAM,English,Spanish
4,US,Oregon,94,Willamette Valley,Willamette Valley,Pinot Noir,Ponzi,105.0,Washington D.C.,North America,NAM,English,Spanish


In [28]:
# Connect to local database
rds_connection_string = "postgres:nbths2014@localhost:5432/etl_project"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [29]:
# Checking Table Names
engine.table_names()

['wine_country']

In [35]:
# SQL Conversion
wine_country.to_sql(name='wine_country', con=engine, if_exists='replace', index=False)

In [36]:
pd.read_sql_query('select * from wine_country', con=engine).head()

,Country,Province,Points,Region_1,Region_2,Variety,Winery,Price,Capital,Continent,Region,Primary Language,Secondary Language
0,US,Oregon,95,Chehalem Mountains,Willamette Valley,Pinot Noir,Bergström,65.0,Washington D.C.,North America,NAM,English,Spanish
1,US,California,95,Diamond Mountain District,Napa,Cabernet Sauvignon,Hall,325.0,Washington D.C.,North America,NAM,English,Spanish
2,US,Oregon,94,Willamette Valley,Willamette Valley,Pinot Noir,Ponzi,105.0,Washington D.C.,North America,NAM,English,Spanish
3,US,Oregon,94,Willamette Valley,Willamette Valley,Pinot Noir,Ponzi,105.0,Washington D.C.,North America,NAM,English,Spanish
4,US,Oregon,94,Willamette Valley,Willamette Valley,Pinot Noir,Ponzi,105.0,Washington D.C.,North America,NAM,English,Spanish
